In [6]:
from datetime import datetime
from elasticsearch import Elasticsearch
es = Elasticsearch([{'host': '10.191.1.241', 'port': 9200}])

In [7]:
es.indices.get_alias("*")

{u'.apm-agent-configuration': {u'aliases': {}},
 u'.kibana_1': {u'aliases': {u'.kibana': {}}},
 u'.kibana_task_manager_1': {u'aliases': {u'.kibana_task_manager': {}}},
 u'tweets-descartar-2020.02.28': {u'aliases': {}},
 u'tweets-descartar-2020.02.29': {u'aliases': {}},
 u'tweets-descartar-2020.03.01': {u'aliases': {}},
 u'tweets-descartar-2020.03.02': {u'aliases': {}},
 u'tweets-descartar-2020.03.03': {u'aliases': {}},
 u'tweets-descartar-2020.03.04': {u'aliases': {}},
 u'tweets-enriquecer-2020.02.28': {u'aliases': {}},
 u'tweets-enriquecer-2020.02.29': {u'aliases': {}},
 u'tweets-enriquecer-2020.03.01': {u'aliases': {}},
 u'tweets-enriquecer-2020.03.02': {u'aliases': {}},
 u'tweets-enriquecer-2020.03.03': {u'aliases': {}},
 u'tweets-enriquecer-2020.03.04': {u'aliases': {}}}

In [21]:
es.indices.stats(index="tweets-enriquecer-*")

{u'_all': {u'primaries': {u'completion': {u'size_in_bytes': 0},
   u'docs': {u'count': 79677, u'deleted': 6526},
   u'fielddata': {u'evictions': 0, u'memory_size_in_bytes': 5143972},
   u'flush': {u'periodic': 0, u'total': 5, u'total_time_in_millis': 0},
   u'get': {u'current': 0,
    u'exists_time_in_millis': 0,
    u'exists_total': 0,
    u'missing_time_in_millis': 0,
    u'missing_total': 0,
    u'time_in_millis': 0,
    u'total': 0},
   u'indexing': {u'delete_current': 0,
    u'delete_time_in_millis': 0,
    u'delete_total': 0,
    u'index_current': 0,
    u'index_failed': 0,
    u'index_time_in_millis': 2597,
    u'index_total': 2129,
    u'is_throttled': False,
    u'noop_update_total': 0,
    u'throttle_time_in_millis': 0},
   u'merges': {u'current': 0,
    u'current_docs': 0,
    u'current_size_in_bytes': 0,
    u'total': 70,
    u'total_auto_throttle_in_bytes': 125829120,
    u'total_docs': 240780,
    u'total_size_in_bytes': 340366207,
    u'total_stopped_time_in_millis': 0,


In [25]:
es.count(index="tweets-enriquecer-*", body=
{
    "query" : {
        "query_string" : {
            "query" : "*speroni"
        }
    }
})

{u'_shards': {u'failed': 0, u'skipped': 0, u'successful': 6, u'total': 6},
 u'count': 2}

In [26]:
es.search(index="tweets-enriquecer-*", body=
{
    "query" : {
        "query_string" : {
            "query" : "*speroni"
        }
    }
})

{u'_shards': {u'failed': 0, u'skipped': 0, u'successful': 6, u'total': 6},
 u'hits': {u'hits': [{u'_id': u'1235031578671120384',
    u'_index': u'tweets-enriquecer-2020.03.04',
    u'_score': 1.0,
    u'_source': {u'@timestamp': u'2020-03-04T02:37:19.842Z',
     u'author': {u'author_account_creation': u'2010-06-11T17:22:22.000Z',
      u'author_description': u'Las opiniones ac\xe1 vertidas as\xed como cada decisi\xf3n que tomo en mi vida son \xfanicamente de m\xed responsabilidad y me hago cargo.',
      u'author_followers_count': 2961,
      u'author_friends_count': 1982,
      u'author_id': u'154590512',
      u'author_location': u'Uy',
      u'author_name': u'Leo en redes \u303d\ufe0f',
      u'author_pseudo': u'LeoEnRedes'},
     u'favorite_count': 0,
     u'is_a_retweet': u'false',
     u'pipeline': {u'kafka_topic': u'tweets-enriquecer',
      u'listener_origin': u'INTENDENCIA'},
     u'retweet_count': 0,
     u'tweet_content': u'@danielcarranza @raulsperoni Una excelente y practi